In [1]:
%matplotlib inline
import dataset
import pandas as pd
import matplotlib.pylab as plt

In [2]:
# Get data
db_url = "sqlite:///data/tripadvisor.db"
db = dataset.connect(db_url)
attractions = db["attractions"]

df = pd.DataFrame([row for row in attractions.all()])

In [3]:
# Add county where missing
gotland_places = [u'Dalhem', u'Tofta', u'Visby', u'Fårö', u'Gotland', u'Fårösund', u'Klintehamn', u'Burgsvik', u'Rydal', u'Roma',
       u'Tingst\xe4de', u'Västergarn']
kalmar_places = [u'Öland', u'Byxelkrok', u'Borgholm', u'Färjestaden', u'Löttorp', u'Mörbylånga']
vasternorrland_places = [u'Örnsköldsvik',u'Ulvöhamn']

df.loc[df.city.isin(gotland_places), 'county'] = 'Gotland'
df.loc[df.city.isin(kalmar_places), 'county'] = u'Kalmar län'
df.loc[df.city.isin(vasternorrland_places), 'county'] = u'Västernorrlands län'


In [4]:
# Cleaning

# Remove arrow from county name
df['county'] = df.loc[:, 'county'].str.replace(u"›","")

In [5]:
def get_score(x):
    score = 0
    for points, perc in enumerate(x):
        score += (points + 1) * perc
    return score

In [6]:
grade_cols = [u'Dåligt', u'Hemskt', u'Medelmåttigt', u'Mycket bra', u'Utmärkt']
grade_cols_perc = ["%s (%%)" % x for x in grade_cols]

# Count percentages
_df = df[grade_cols].apply(lambda x: x / x.sum(), axis=1)
_df.columns = grade_cols_perc
df = df.join(_df)

# Count score
df[u"Poäng"] = df[grade_cols_perc].apply(get_score, axis=1)

# Add column with total number of reviews
df["Antal recensioner"] = df[grade_cols].sum(axis=1)

# Remove attractions with few reviews
# Attractions with fewer than 5 reviews tend to have a pretty distored score distribution
LIMIT = 5
df_filt = df[df["Antal recensioner"] > LIMIT]

print "Removed attractions with fewer than %s reviews. From %s to %s rows." % (LIMIT, len(df), len(df_filt))

Removed attractions with fewer than 5 reviews. From 2033 to 738 rows.


### Vilket län har hetast attraktioner?

När vi räknar ut ett medelpoäng för alla attraktioner som fått minst tio recensioner ser topplistan ut så här.

In [7]:
by_county = df_filt.groupby("county")[grade_cols_perc].mean()
by_county[u"Poäng"] = by_county.apply(get_score, axis=1)
by_county[u"Antal attraktioner"] = df_filt.groupby("county").count().id
by_county.sort_values(u"Poäng", ascending=False)


,Dåligt (%),Hemskt (%),Medelmåttigt (%),Mycket bra (%),Utmärkt (%),Poäng,Antal attraktioner
county,,,,,,,
Bohuslän,0.000000,0.000000,0.000000,0.385714,0.614286,4.614286,2
Norrbottens län,0.026734,0.011239,0.066893,0.296679,0.598454,4.428879,32
Västerbottens län,0.013317,0.011905,0.093628,0.338160,0.542990,4.385601,12
Västernorrlands län,0.010110,0.002083,0.070535,0.431923,0.485349,4.380318,20
Gotland,0.008309,0.015560,0.073390,0.393797,0.508944,4.379506,17
Gävleborgs län,0.026067,0.020614,0.092159,0.354183,0.506976,4.295388,19
Kronobergs län,0.019944,0.005952,0.129647,0.354580,0.489876,4.288491,14
Östergötlands län,0.011238,0.004875,0.119344,0.415296,0.449248,4.286442,33
Värmlands län,0.032099,0.011667,0.095076,0.373601,0.487557,4.272849,14


Vi kan strunta i Bohuslän som bara har en attraktion med här. Då är **Västerbotten**, **Jönköping** och **Värmland** de län vars besöksmål får högst betyg på Tripadvisor.

Vilka attraktioner är då det?

In [33]:
columns_to_include = ["city","name", u"Poäng", "Antal recensioner"]
df_filt[df_filt.county == u"Norrbottens län"][columns_to_include].sort_values(u"Poäng", ascending=False).head()

,city,name,Poäng,Antal recensioner
1927,Harads,Pure Lapland - Tours,5.0,10.0
1900,Vidsel,Laponie Mush - Day Tours,5.0,16.0
1899,Norrfjärden,Svedjekojan in Swedish Lapland,5.0,8.0
1434,Jukkasjarvi,Ice Driving,5.0,7.0
1229,Arvidsjaur,Burning Snow,5.0,9.0


In [34]:
df_filt[df_filt.county == u"Västerbottens län"][columns_to_include].sort_values(u"Poäng", ascending=False).head()

,city,name,Poäng,Antal recensioner
1690,Vilhelmina,Wilderness Latitude - Day Tour,5.000000,9.0
1734,Vindeln,Aurora Borealis Adventures,5.000000,13.0
1689,Vilhelmina,Fatmomakke Kyrkstad,4.625000,8.0
958,Umeå,Guitars the Museum,4.576271,59.0
960,Umeå,Bildmuseet,4.516129,31.0


In [35]:
df_filt[df_filt.county == u"Västernorrlands län"][columns_to_include].sort_values(u"Poäng", ascending=False).head()

,city,name,Poäng,Antal recensioner
1392,Docksta,Via Ferrata,4.800000,10.0
1262,Örnsköldsvik,Slattdalsskrevan,4.777778,9.0
843,Sundsvall,Sundsvall-Timrå Turistbyrå,4.714286,7.0
1263,Örnsköldsvik,Världsarvet Höga Kusten,4.666667,6.0
1391,Docksta,Skuleberget,4.500000,28.0


## Vilken stad har hetast attraktioner?

Vi räknar medelbetyget på attrakioner i varje stad och inkluderar bara städer som har minst fem attraktioner.

In [11]:
# Get mean reviews
avg_reviews = df_filt.groupby("city").mean()[grade_cols_perc]

# Get scores
avg_reviews[u"Poäng"] = avg_reviews[grade_cols_perc].apply(get_score, axis=1)

# Get number of attraktions
avg_reviews["Antal attraktioner"] = df_filt.groupby("city").count()["id"]

# Filter cities with few attractions
avg_reviews[avg_reviews["Antal attraktioner"] > 5].sort_values(u"Poäng", ascending=False)

,Dåligt (%),Hemskt (%),Medelmåttigt (%),Mycket bra (%),Utmärkt (%),Poäng,Antal attraktioner
city,,,,,,,
Ängelholm,0.000000,0.000000,0.049325,0.332313,0.618363,4.569038,6
Vimmerby,0.003086,0.000000,0.093474,0.311596,0.591843,4.489109,6
Luleå,0.003247,0.000000,0.079004,0.371985,0.545764,4.457019,7
Varberg,0.016667,0.013889,0.083333,0.303704,0.582407,4.421296,6
Visby,0.005034,0.011111,0.080130,0.416425,0.487300,4.369845,10
Borås,0.044901,0.000000,0.082538,0.339397,0.533164,4.315923,7
Sundsvall,0.028885,0.005952,0.076385,0.399222,0.489555,4.314609,7
Norrköping,0.015563,0.000000,0.124881,0.406021,0.453536,4.281966,10
Linköping,0.002551,0.010417,0.106574,0.477556,0.402902,4.267841,8


## Vilken typ av attraktioner får högst betyg?

In [12]:
# Aggregate scores by tag
tags = {}
        
for row in df_filt[["tags", u"Poäng"]].iterrows():
    score = row[1][1]
    for tag in row[1][0].split("|"):
        if tag not in tags:
            tags[tag] = []
        # Append score
        tags[tag].append(score)


In [13]:
# Create a dataframe with mean score by tag
by_tag = pd.DataFrame({
    "Medelbetyg": [pd.Series(x).mean() for x in tags.values()],
    "Antal aktiviteter": [len(x) for x in tags.values()],
}, index=tags.keys())


Vi räknar ut ett medelbetyg för alla aktivitetstyper som förekommer minst fem gånger.

In [14]:
by_tag[by_tag["Antal aktiviteter"] > 5].sort_values("Medelbetyg", ascending=False)

,Antal aktiviteter,Medelbetyg
Sightseeing-turer,8,4.593370
Vattensport och rundturer med båt,6,4.573571
Rundturer,31,4.564725
Andra friluftsaktiviteter,38,4.504051
Natur- och vildmarksområden,25,4.491434
Nationalparker,13,4.454623
Öar,14,4.407306
Kyrkor och katedraler,26,4.394396
Friluftsaktiviteter,91,4.375256
Historiska museer,32,4.307848


## Vilka aktiviteter karaktäriserar länen?


In [24]:
# Get a list of tags for every county
counties = {}
        
for row in df_filt[["tags", "county", u"Poäng"]].iterrows():
    county = row[1][1]
    score = row[1][2]
    for tag in row[1][0].split("|"):
        if county not in counties:
            counties[county] = []
        # Append score
        counties[county].append((tag, score))

Vilka är de vanligaste aktivitetstyperna i varje län?

In [37]:
# Print  list of the 
for county, _tags in counties.iteritems():
    _df = pd.DataFrame({
            "tag": [x[0] for x in _tags],
            u"Poäng": [x[1] for x in _tags],
        })
    
    tags = _df.groupby("tag").mean()
    tags["Antal aktiviteter"] = _df.groupby("tag").count()

    print county
    print "==================="
    print tags.sort_values("Antal aktiviteter", ascending=False).head(10)
    print "\n\n"


Skåne län
                                       Poäng  Antal aktiviteter
tag                                                            
Sevärdheter och landmärken          4.127923                 43
Museer                              4.306545                 29
Natur och parker                    4.447673                 25
Specialmuseer                       4.325523                 21
Intressanta platser och landmärken  4.081162                 19
Friluftsaktiviteter                 4.584292                 15
Mer                                 4.391768                  8
Slott                               4.370234                  7
Parker                              4.328548                  7
Natur- och vildmarksområden         4.584187                  5



Västernorrlands län
                                       Poäng  Antal aktiviteter
tag                                                            
Natur och parker                    4.428275                  4
Friluft

## Vilka aktiviteter karakätiersera del olika städerna?



In [39]:
# Get a list of tags for every county
cities = {}
        
for row in df_filt[["tags", "city", u"Poäng"]].iterrows():
    city = row[1][1]
    score = row[1][2]
    for tag in row[1][0].split("|"):
        if city not in cities:
            cities[city] = []
        # Append score
        cities[city].append((tag, score))

In [40]:
# Print  list of the 
for city, _tags in cities.iteritems():
    _df = pd.DataFrame({
            "tag": [x[0] for x in _tags],
            u"Poäng": [x[1] for x in _tags],
        })
    
    tags = _df.groupby("tag").mean()
    tags["Antal aktiviteter"] = _df.groupby("tag").count()
    if tags["Antal aktiviteter"].sum() > 10:
        print city
        print "==================="
        print tags.sort_values("Antal aktiviteter", ascending=False).head(10)
        print "\n\n"

Solna
                               Poäng  Antal aktiviteter
tag                                                    
Sevärdheter och landmärken  3.774155                  3
Historiska platser          4.041667                  2
Arenor och idrottsarenor    3.239130                  1
Parker                      4.363636                  1
Span                        3.142857                  1
Spa och välmående           3.142857                  1
Slott                       3.750000                  1
Shopping                    3.989362                  1
Rundturer                   4.333333                  1
Natur och parker            4.363636                  1



Järbo
                               Poäng  Antal aktiviteter
tag                                                    
Friluftsaktiviteter         4.416667                  2
Andra friluftsaktiviteter   4.833333                  1
Ekoturer                    4.833333                  1
Längdskidåkningsområden     4.000

In [ ]:
# Should we trust activities with few reviews
# Visulize the distribution of scores
bins = [0, 2, 4, 6, 8, 10, 15, 1000]
bins = [0,5,1000]
df["Antal recensioner - grupp"] = pd.cut(df["Antal recensioner"], bins=bins)

df[u"Poäng"].hist(by=df["county"])